# Setup

In [ ]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

This code performs several package management and environment setup tasks:

The first line `%%capture` suppresses output messages in a Jupyter/Colab notebook environment.

The next section manipulates Python's module system:
```python
import sys
modules = list(sys.modules.keys())
for x in modules:
    sys.modules.pop(x) if "PIL" in x or "google" in x else None
```
This code removes any modules containing "PIL" (Python Imaging Library) or "google" in their names from the currently loaded modules. This technique can help avoid conflicts when reinstalling or upgrading these packages.

The subsequent lines install specific packages using pip:
```python
!pip install unsloth vllm
!pip install --upgrade pillow
```
`unsloth` and `vllm` are machine learning optimization libraries, while `pillow` is an imaging library being upgraded to its latest version.

The commented lines provide optional installation instructions:
```python
# !pip install diffusers
```
This shows that the `diffusers` library would need manual installation when running on a local machine rather than in Colab.

The final line installs a specific version of the TRL (Transformer Reinforcement Learning) library directly from GitHub:
```python
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b
```
The long string after @ represents a specific commit hash, ensuring installation of a precise version of the library. This technique helps maintain consistency and avoid potential issues with newer versions.

The code demonstrates environment preparation for a machine learning project, specifically setting up dependencies for working with transformer models and image processing.

In [ ]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-13 09:26:16 __init__.py:190] Automatically detected platform cuda.


This code is setting up a specialized language model optimization system. Let's break it down step by step:

The first line imports two key components from the unsloth library:
```python
from unsloth import FastLanguageModel, PatchFastRL
```
`FastLanguageModel` is a class that provides accelerated implementations of language models. Think of it as a tuned racing engine compared to a standard car engine - it's optimized for speed while doing the same fundamental job.

`PatchFastRL` is a function that modifies or "patches" existing machine learning code to incorporate reinforcement learning techniques. Reinforcement learning is like training a dog - the model learns from rewards and penalties rather than just examples.

The second line applies this patch:
```python
PatchFastRL("GRPO", FastLanguageModel)
```
This connects GRPO (Generalized Reward-Penalized Optimization) with the fast language model system. GRPO is an advanced training method that helps models learn more efficiently. Think of it as upgrading a training regimen - instead of just memorizing examples, the model learns to improve its performance based on how well it's doing at its assigned tasks.

The combination of these components creates a system that can train language models more quickly and effectively than traditional methods.

In [ ]:
# Standard library imports
import re
import os

# Third-party library imports
import torch
from datasets import load_dataset, Dataset
from google.colab import userdata
from huggingface_hub import notebook_login, login
from trl import GRPOConfig, GRPOTrainer
from unsloth import is_bfloat16_supported
from vllm import SamplingParams

Let's break this code down into logical sections to understand how it sets up a sophisticated machine learning environment.

Starting with the standard library imports:
```python
import re   # Regular expressions for text pattern matching
import os   # Operating system interface for file/directory operations
```
These provide essential Python functionality that will be needed for text processing and file handling.

Moving to the specialized machine learning imports:
```python
import torch  # The core deep learning framework
```
PyTorch serves as the foundation for all the neural network operations that will follow. It provides the mathematical tools needed to build and train AI models.

Next, we have data management tools:
```python
from datasets import load_dataset, Dataset
```
These imports from the Hugging Face datasets library let us work with large-scale datasets efficiently. `load_dataset` fetches pre-made datasets, while `Dataset` helps create custom ones.

For authentication and cloud integration:
```python
from google.colab import userdata
from huggingface_hub import notebook_login, login
```
These tools handle credentials and authentication. The Colab integration manages Google-specific features, while the Hugging Face imports allow secure access to their model hub and resources.

The training infrastructure comes from:
```python
from trl import GRPOConfig, GRPOTrainer
```
TRL (Transformer Reinforcement Learning) provides the GRPO (Generalized Reward-Penalized Optimization) framework. This is advanced machinery for fine-tuning language models with reinforcement learning techniques.

Hardware optimization tools:
```python
from unsloth import is_bfloat16_supported
```
This checks whether your hardware supports bfloat16, a specialized number format that can make training faster and more memory-efficient while maintaining good precision.

Finally, for model output control:
```python
from vllm import SamplingParams
```
VLLM provides tools to manage how the model generates text, controlling aspects like randomness and output length.

Together, these imports create a comprehensive environment for advanced language model training, combining data handling, model optimization, cloud integration, and output generation control. It's like assembling all the specialized tools needed in a high-tech workshop before beginning a complex project.

In [ ]:
class CFG:
  model = "Qwen/Qwen2.5-3B-Instruct"
  lora_rank = 64
  max_seq_length = 1024
  temperature = 0.8
  top_p = 0.95
  max_tokens = 1024

The `CFG` class defines the core configuration settings for training and running a large language model. Let's examine each setting and understand its purpose:

```python
model = "Qwen/Qwen2.5-3B-Instruct"
```
This specifies the base model to be used - Qwen 2.5, a 3 billion parameter model designed for instruction following. The size of 3B represents a medium-sized model that balances performance with computational requirements.

```python
lora_rank = 64
```
This setting controls the complexity of the LoRA (Low-Rank Adaptation) fine-tuning approach. A rank of 64 means the model will learn 64 different patterns of adaptations. Think of it like having 64 different ways to modify the original model's knowledge. Higher ranks allow more complex adaptations but require more memory and computation.

```python
max_seq_length = 1024
```
This defines how many tokens (pieces of text) the model can process at once - in this case, 1024 tokens. Each token might be a word or part of a word. For perspective, this is roughly equivalent to 750-800 words of English text. Text longer than this will need to be processed in chunks.

```python
temperature = 0.8
```
Temperature controls how "creative" or random the model's outputs are. At 0.8, it strikes a balance between deterministic (perfectly repeatable) responses and creative variation. Lower values like 0.2 would make outputs more predictable, while higher values like 1.5 would make them more random.

```python
top_p = 0.95
```
This is the nucleus sampling parameter, which controls how many different word choices the model considers at each step. A value of 0.95 means it will consider enough options to cover 95% of the probability mass. This helps maintain coherent outputs while allowing some variety.

```python
max_tokens = 1024
```
This limits how long the model's responses can be, measured in tokens. Setting this equal to `max_seq_length` means the model can generate responses up to the maximum length it can process at once.

These settings work together to shape how the model learns and generates text. The configuration balances resource usage (through model size and sequence length), learning capacity (through LoRA rank), and output characteristics (through temperature and top_p).

In [ ]:
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

This line configures an environment variable that affects how the Hugging Face Hub transfers files. Let me explain what's happening and why it matters.

The line `os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"` sets an environment variable that switches on Hugging Face's custom file transfer system. Environment variables are like global settings that can influence how programs behave.

To understand why this matters, let's look at what happens when you download models from Hugging Face. Normally, when you request a model, it downloads using standard HTTP protocols - think of this as taking the regular roads. But Hugging Face has built a specialized highway system (their custom transfer protocol) that can be more efficient, especially for large files like machine learning models.

When you set this variable to "1", you're telling your system "use the specialized Hugging Face transfer system instead of the standard one." This can lead to several benefits:

1. Faster downloads: The custom protocol is optimized specifically for large model files
2. Better reliability: It includes features like automatic retries and resume capabilities
3. Improved efficiency: It can better handle the specific way model files are structured


In [ ]:
login(token = userdata.get('HF_TOKEN'))

This line handles authentication with the Hugging Face Hub, allowing secure access to models and datasets. Let's break down what's happening step by step.

The `login()` function comes from the Hugging Face Hub library and establishes a connection between your code and Hugging Face's servers. Think of it like showing your ID card at a secure building - you need to prove you have permission to access the resources inside.

The `userdata.get('HF_TOKEN')` part retrieves your personal access token from Google Colab's secure storage system. This token is like a digital key that proves your identity to Hugging Face. Having it stored in Colab's `userdata` system keeps it secure - it's similar to how a password manager safely stores your passwords instead of writing them down where others might see them.

When you create a Hugging Face account, you generate this token from your account settings. The token is a long string of characters that uniquely identifies you and your permissions. When you save it in Colab's `userdata`, you're storing it in a secure place that only your notebooks can access.

The whole line combines these elements: it takes your securely stored token and uses it to log into Hugging Face's systems. Once this authentication succeeds, your code will have permission to:
- Download private models
- Access gated or restricted content
- Push changes to repositories you have access to
- Use Hugging Face's API services

If the login fails (for example, if the token is invalid or expired), you would need to generate a new token from the Hugging Face website and update it in your Colab userdata. This security system helps ensure that only authorized users can access and modify resources on the Hugging Face Hub.

# Functions

In [ ]:

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()


This function processes text that contains XML-style tags to extract answers marked with specific tags. Let's break down how it works and understand its purpose.

The function takes a string parameter called `text` and returns another string. The type hints (`: str` and `-> str`) make it clear that both the input and output are text strings. These type hints help other developers understand the function's requirements and make the code easier to maintain.

Let's examine the extraction process:

```python
answer = text.split("<answer>")[-1]
```
This line splits the text at every occurrence of `<answer>` and takes the last piece. If we had text like "Here is a <answer>42</answer>", splitting would give us ["Here is a ", "42</answer>"], and we take "42</answer>". Using [-1] ensures we get the last answer if there happen to be multiple answer tags.

```python
answer = answer.split("</answer>")[0]
```
This line splits at the closing tag and takes the first piece. Continuing our example, "42</answer>" would split into ["42", ""], and we take "42". Taking [0] ensures we get just the answer content.

```python
return answer.strip()
```
Finally, we remove any extra whitespace from the beginning and end of our extracted answer using `strip()`. This ensures clean, consistent output even if there were spaces around the answer in the original text.

This function would be particularly useful in processing responses from language models or other systems that structure their output using XML-style tags. For example, if you had text like:
```
Some preliminary text here
<answer>
   This is the important part we want to extract
</answer>
Other text we don't need
```
The function would return "This is the important part we want to extract" without the surrounding tags or extra spaces.

One limitation to note is that this function assumes well-formed XML with exactly one set of answer tags. If the tags are malformed or missing, the function might raise an error. In a production environment, you might want to add error handling for such cases.

In [ ]:
def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

This function extracts answers marked with hash symbols (####) from text, offering an alternative extraction method to the XML-based approach. Let's examine how it works and understand its design choices.

The function's type hints tell us something interesting: it accepts a string (`text: str`) but can return either a string or None (`-> str | None`). This union type hint signals that the function might not always find an answer to extract.

The first line performs a crucial check:
```python
if "####" not in text:
    return None
```
This defensive programming approach immediately handles cases where no hash markers exist. Rather than trying to process text without markers and potentially causing errors, it gracefully returns None. Think of it like a security guard checking if you have the right credentials before letting you into a building.

If we pass this check, the extraction happens in a single line:
```python
return text.split("####")[1].strip()
```
This line does several operations in sequence. Let's break it down:
1. `text.split("####")` divides the text at each occurrence of four hash symbols
2. `[1]` takes the second piece (index 1) of the split text
3. `strip()` removes any extra whitespace from the beginning and end

For example, if we had text like:
```
Here is a question
#### This is the answer we want ####
More text after
```
The split would create: ["Here is a question", " This is the answer we want ", "More text after"]
Taking index [1] gives us " This is the answer we want "
And strip() cleans it up to "This is the answer we want"

One important detail: this function only extracts the text after the first set of hash symbols. If there are multiple answers marked with ####, it will only return the first one. This behavior differs from the XML extractor we saw earlier, which took the last answer when multiple were present.

The function also assumes that anything between sets of #### is the answer, making it less structured but potentially more flexible than the XML approach. This could be useful when working with language models that might find it easier to use simple hash markers rather than properly nested XML tags.

In [ ]:
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

This function retrieves and processes mathematical questions from the GSM8K dataset, which is a collection of grade school math problems. Let me walk you through how it works and why each part matters.

The function takes one parameter called `split` that defaults to "train", allowing us to choose whether we want training or testing data. The return type hint `-> Dataset` tells us it will give back a Dataset object, which is a specialized data structure from the Hugging Face datasets library.

Let's look at the data loading step:
```python
data = load_dataset('openai/gsm8k', 'main')[split]
```
This loads the GSM8K dataset from OpenAI. The 'main' parameter specifies we want the primary version of the dataset, and `[split]` selects either the training or testing portion. Think of this like going to a library and asking for either the practice problems (train) or the final exam questions (test).

The transformation of the data happens in the mapping step:
```python
data = data.map(lambda x: {
    'prompt': [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': x['question']}
    ],
    'answer': extract_hash_answer(x['answer'])
})
```
This code transforms each question-answer pair into a format suitable for language model training. Let's break down what's happening:

1. The `map` function applies our transformation to every item in the dataset, like an assembly line worker performing the same task on each piece.

2. For each item, we create a new dictionary with two keys:
   - `prompt`: An array of messages formatted for chat-style interaction:
     - A system message containing `SYSTEM_PROMPT` (which sets the context for the model)
     - A user message containing the actual math question
   - `answer`: The extracted answer using our `extract_hash_answer` function from earlier

The `# type: ignore` comments tell type checkers to skip checking these lines. This is often needed when working with dynamic data structures like datasets where the type system might not fully capture the complexity of the operations.

For example, if we had this input:
```python
{
    'question': 'If John has 5 apples and gives 2 away, how many does he have left?',
    'answer': 'Let\'s solve this step by step:\n1) Start with 5 apples\n2) Give away 2 apples\n#### 3 ####'
}
```

The function would transform it into:
```python
{
    'prompt': [
        {'role': 'system', 'content': 'You are a helpful math tutor...'},
        {'role': 'user', 'content': 'If John has 5 apples and gives 2 away, how many does he have left?'}
    ],
    'answer': '3'
}
```

This structured format makes it easier to train language models to understand and solve mathematical problems while maintaining a clear separation between questions and answers. The chat-style format also helps the model understand its role in the conversation and provide more appropriate responses.

In [ ]:

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]


These functions evaluate and score model responses based on different criteria. Let's explore how each reward function works and what it rewards.

The first function, `correctness_reward_func`, evaluates whether answers match the expected solution:
```python
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
```
It extracts the actual response from the model's output using XML tags and prints key debugging information like the question, expected answer, full response, and extracted answer. The function awards 2.0 points for exact matches and 0.0 for incorrect answers. This strict scoring encourages precise answers.

Next, `int_reward_func` checks if responses are numerical:
```python
def int_reward_func(completions, **kwargs) -> list[float]:
```
This function awards 0.5 points if the extracted answer contains only digits. For example, "42" would earn points, but "forty-two" or "42.5" would not. This encourages the model to provide numerical answers in a standardized format.

The `strict_format_reward_func` enforces a rigid XML structure:
```python
def strict_format_reward_func(completions, **kwargs) -> list[float]:
```
It uses a regular expression pattern to check if responses follow this exact format:
```
<reasoning>
[explanation text]
</reasoning>
<answer>
[answer text]
</answer>
```
The `^` and `$` in the pattern require the format to match the entire response, with no extra text before or after. The function awards 0.5 points for properly formatted responses.

Finally, `soft_format_reward_func` provides a more lenient format check:
```python
def soft_format_reward_func(completions, **kwargs) -> list[float]:
```
Its regular expression allows more flexibility:
- Tags don't need to be on separate lines
- Whitespace between tags can vary
- Extra text can appear before or after the tags

Together, these reward functions create a scoring system that encourages:
1. Correct answers (highest reward of 2.0)
2. Proper formatting (0.5 points)
3. Numerical responses (0.5 points)

By combining these rewards, the training process guides the model toward producing responses that are not only correct but also consistently formatted and properly structured. The debugging output from `correctness_reward_func` helps monitor the training process and identify any patterns in incorrect responses.

In [ ]:

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Let's examine these two connected functions that evaluate how well responses follow a specific XML format structure, with a sophisticated scoring system that accounts for both format correctness and extra content.

The `count_xml` function performs a detailed analysis of XML tag placement and spacing, returning a score between 0 and 0.5. Let's break down how it evaluates each component:

1. Opening reasoning tag check:
```python
if text.count("<reasoning>\n") == 1:
    count += 0.125
```
This awards 0.125 points for having exactly one reasoning tag followed by a newline. This enforces the tag appears once and starts a new line.

2. Closing reasoning tag check:
```python
if text.count("\n</reasoning>\n") == 1:
    count += 0.125
```
Similarly, this checks for one closing reasoning tag with newlines on both sides, ensuring proper tag isolation.

3. Opening answer tag check:
```python
if text.count("\n<answer>\n") == 1:
    count += 0.125
    count -= len(text.split("\n</answer>\n")[-1])*0.001
```
This part does two things:
- Awards 0.125 points for proper answer tag placement
- Subtracts a small penalty (0.001) for each character that appears after the final answer tag. This encourages concise responses without trailing content.

4. Closing answer tag check:
```python
if text.count("\n</answer>") == 1:
    count += 0.125
    count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
```
This applies the same scoring logic but adjusts the penalty calculation to account for the different tag format.

The companion function `xmlcount_reward_func` applies this scoring to model outputs:
```python
def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]
```
It extracts the content from each completion and applies the scoring function, creating a list of scores that reflect how well each response follows the desired format.

This scoring system shows sophisticated design choices:
- The total positive score (0.5) matches other format reward functions, maintaining consistency
- Breaking the score into four equal parts (0.125 each) emphasizes the importance of each tag element
- The small penalties for extra content create a gentle pressure toward cleaner outputs without harshly punishing minor formatting issues
- The newline requirements ensure clear visual separation between components, making outputs more readable

Together, these functions create a nuanced reward system that encourages clean, well-structured responses while being tolerant enough to allow the model to learn gradually rather than requiring immediate perfection.

# Model prep

In [ ]:

max_seq_length = CFG.max_seq_length # Can increase for longer reasoning traces
lora_rank = CFG.lora_rank # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CFG.model,
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.48%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 288.
Unsloth: vLLM's KV Cache can use up to 17.15 GB. Also swap space = 6 GB.
INFO 02-13 09:26:46 config.py:542] This model supports multiple tasks: {'reward', 'embed', 'classify', 'generate', 'score'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwa

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

This code sets up and loads a large language model with specific optimizations for performance and memory usage. Let's explore how it works and why each setting matters.

First, we extract two key configuration values:
```python
max_seq_length = CFG.max_seq_length
lora_rank = CFG.lora_rank
```
The sequence length determines how much text the model can process at once. Think of it like the size of a window through which the model views text - a larger window lets it see more context at once. The comment indicates we can increase this for longer reasoning traces, which would be helpful when the model needs to work through complex problems step by step.

The LoRA rank setting influences how much the model can learn and adapt. LoRA (Low-Rank Adaptation) is like teaching an experienced professional new specialized skills - the rank determines how many new patterns they can learn. A higher rank allows more complex adaptations but requires more computational resources, much like how learning multiple skills simultaneously takes more time and energy.

The main model loading happens with this command:
```python
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = CFG.model,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,
)
```

Let's examine each parameter:
- `model_name = CFG.model`: Specifies we're using the Qwen 2.5 3B Instruct model, which is designed specifically for following instructions
- `load_in_4bit = True`: Enables 4-bit quantization, a technique that's like compressing the model's knowledge into a more memory-efficient format. The comment notes we should set this to False if we want to use 16-bit LoRA
- `fast_inference = True`: Activates vLLM's optimized inference system, similar to taking a shortcut that gets you to the same destination faster
- `max_lora_rank = lora_rank`: Sets up the adaptation capacity we discussed earlier
- `gpu_memory_utilization = 0.5`: Tells the system to use only half of the available GPU memory. This conservative setting helps prevent out-of-memory errors, leaving room for other processes and safety margin. The comment helpfully notes we can reduce this if we run into memory issues

The function returns two components:
1. The model itself, which contains all the neural network parameters and knowledge
2. The tokenizer, which converts between human-readable text and the numerical format the model understands, like a translator between human language and machine language

This setup creates a balanced system that prioritizes both performance (through optimizations like 4-bit quantization and fast inference) and flexibility (through LoRA adaptation), while maintaining safeguards against resource exhaustion through careful memory management.

In [ ]:


model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth 2025.2.5 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


This code configures a large language model for efficient fine-tuning using LoRA (Low-Rank Adaptation), a technique that reduces memory usage and training time.

The code starts by getting a PEFT (Parameter Efficient Fine-Tuning) version of the base model using `FastLanguageModel.get_peft_model()`. This creates a modified version that can be trained more efficiently.

The `lora_rank` parameter sets the rank of the low-rank matrices used in LoRA. A lower rank means less memory usage but potentially reduced model capacity, while a higher rank allows more expressive power but uses more memory.

The `target_modules` list specifies which parts of the model will be fine-tuned. These modules handle different aspects of the transformer architecture:
- `q_proj`, `k_proj`, `v_proj`, `o_proj`: Handle attention mechanisms (queries, keys, values, and output projections)
- `gate_proj`: Controls information flow in the network
- `up_proj` and `down_proj`: Handle dimensionality changes in the model

Setting `lora_alpha` equal to `lora_rank` effectively makes the learning rate scaling factor 1.0, which is a common practice in LoRA fine-tuning.

The `use_gradient_checkpointing` parameter set to "unsloth" enables memory-efficient training by recomputing certain values during the backward pass instead of storing them. This allows for training with longer sequences of text.

Finally, `random_state=3407` sets a fixed random seed for reproducibility, ensuring the same initialization of LoRA parameters across different training runs.

The comment "Remove QKVO if out of memory" suggests that if memory issues arise, removing the attention-related modules (`q_proj`, `k_proj`, `v_proj`, `o_proj`) from fine-tuning can reduce memory usage while still allowing meaningful model adaptation through the remaining modules.

# Data Prep

In [ ]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""


dataset = get_gsm8k_questions()


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

This code sets up a structured format for generating responses to GSM8K math questions (a dataset of grade school math word problems) using a step-by-step reasoning approach.

The first string `SYSTEM_PROMPT` defines a template that enforces responses to have two distinct XML-style sections:
1. A `<reasoning>` section where step-by-step thinking is shown
2. An `<answer>` section for the final solution

The `XML_COT_FORMAT` string creates a formatted version of this template using Python's string formatting. The curly braces `{reasoning}` and `{answer}` are placeholders that can be filled in with actual content. This structure implements "Chain of Thought" (COT) prompting - a technique that encourages language models to show their work by breaking down complex problems into smaller steps.

For example, when solving a math problem, the format might be used like this:
```python
formatted_response = XML_COT_FORMAT.format(
    reasoning="1. First, we calculate the cost per item\n2. Then multiply by quantity...",
    answer="The total cost is $42"
)
```

The `get_gsm8k_questions()` function retrieves the dataset of math problems that will be processed using this format. Each question from the dataset would be presented to the model, which would then respond with its reasoning steps and final answer in the structured XML format.

This structured approach serves multiple purposes:
- It makes the model's thinking process transparent and debuggable
- It separates the problem-solving steps from the final answer
- It creates consistency in response format, making it easier to process or evaluate the outputs
- It helps ensure the model doesn't skip steps or jump directly to answers without showing work

The XML tags make it easy to programmatically extract either the reasoning or the answer section for further processing or evaluation.

# Model train

In [ ]:

training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


This code configures training arguments for a language model using a technique called Gradient-based Role Optimization (GRPO). Let me explain each parameter and its significance for the training process.

Starting with inference settings, `use_vllm = True` enables vLLM (very large language model) acceleration, which optimizes memory usage and speeds up the model's predictions during training.

The learning rate of 5e-6 is quite conservative, which helps prevent large parameter updates that could destabilize training. This works together with the Adam optimizer parameters: beta1 (0.9) and beta2 (0.99) control the exponential decay rates for momentum estimation. The higher beta2 value of 0.99 means the optimizer takes a longer view of past gradients, helping smooth out training.

Weight decay of 0.1 adds regularization to prevent overfitting by penalizing large parameter values. The warmup ratio of 0.1 means the first 10% of training steps use gradually increasing learning rates, helping stabilize early training. The "cosine" learning rate scheduler then smoothly decreases the learning rate over time, following a cosine curve.

For memory management, the code uses 8-bit Adam optimization (`adamw_8bit`) to reduce memory usage. It automatically chooses between bfloat16 and fp16 precision based on hardware support - bfloat16 offers better numerical stability when available.

The batch size configuration keeps memory usage low: one example per device with no gradient accumulation (though the comment suggests increasing to 4 for smoother training). The model will generate 8 completions per input, but this can be reduced if memory becomes constrained.

The sequence length limits are set to reasonable values: 256 tokens for prompts and 200 for completions. This balances between having enough context for good performance while managing memory usage.

The gradient clipping norm of 0.1 prevents exploding gradients by scaling down large gradients, adding stability to training. The model saves its progress to an "outputs" directory, and while it's configured not to use external logging ("none"),  Weights & Biases could be enabled for monitoring training progress.

These settings aim to balance between training stability, memory efficiency, and computational speed while allowing for easy adjustment based on available resources and specific training needs.

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 119,734,272


-------------------- Question:
Ahmed and Emily are having a contest to see who can get the best grade in the class. There have been 9 assignments and Ahmed has a 91 in the class. Emily has a 92. The final assignment is worth the same amount as all the other assignments. Emily got a 90 on the final assignment. What is the minimum grade Ahmed needs to get to beat Emily if all grades are whole numbers? 
Answer:
100 
Response:
<reasoning>
To find the minimum grade Ahmed needs to get to beat Emily, we first need to consider the total points both Ahmed and Emily have accumulated. Since there are 9 assignments in total and each assignment is worth the same amount, the difference in total points between Ahmed and Emily before the final assignment is calculated. Then, we'll consider how the final assignment impacts their total scores. If Emily scored 90 on her final assignment, we determine what score Ahmed needs on his final assignment to surpass Emily.

The total points for 9 assignments are 

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.125000,0.000000,200.000000,0.000000
2,-0.000000,0.076375,0.137532,200.000000,0.000000
3,0.000000,-0.147250,0.174998,187.625000,0.000462
4,0.000000,0.125000,0.000000,200.000000,0.000342
5,0.000000,0.125000,0.000000,200.000000,0.000299
6,0.000000,-0.085875,0.164324,174.750000,0.000439
7,0.000000,-0.217125,0.223099,194.500000,0.000768
8,0.000000,-0.039125,0.218415,194.750000,0.000731
9,0.000000,-0.099250,0.241063,184.750000,0.000574
10,0.000000,-0.010875,0.123517,153.875000,0.000505


Streaming output truncated to the last 5000 lines.
Therefore, Grace’s tower is 35 inches taller than Clyde’s tower 
Extracted:
<reasoning>
First, we need to determine the height of Clyde's tower. Let's denote the height of Clyde's tower as \( h \). According chord Grace's tower is 8 times the height of Clyde's tower, and we are told that the total height of Grace's tower is 40 inches. Therefore, we can establish the following relationship:

\[ 8 \times h = 40 \]

Solving for \( h \), we divide both sides of the equation by 8:

\[ h = \frac{40}{8} = 5 \]

So, Clyde's tower is 5 inches tall. 

Next, we need to determine how many inches taller Grace's tower is compared to Clyde's. To do this, we simply subtract the height of Clyde's tower from the height of Grace's tower:

\[ 40 - 5 = 35 \]

Therefore, Grace’s tower is 35 inches taller than Clyde’s tower
-------------------- Question:
Andrew's father buys a package of 100 masks. Andrew lives with his 2 parents and 2 siblings. All members 

TrainOutput(global_step=250, training_loss=0.0004522450193435361, metrics={'train_runtime': 1636.3505, 'train_samples_per_second': 0.153, 'train_steps_per_second': 0.153, 'total_flos': 0.0, 'train_loss': 0.0004522450193435361})

This code initializes and runs a specialized trainer that uses Gradient-based Role Optimization (GRPO) to fine-tune a language model. Let me walk you through how this training setup works.

The trainer takes several key components that work together during training. First, it needs the model we want to train and a tokenizer (called `processing_class` here) that converts text into numbers the model can understand.

The heart of this training approach lies in the `reward_funcs` - these are functions that evaluate how well the model is doing and guide its improvement. Let's look at what each reward function does:

1. `xmlcount_reward_func`: Ensures the model produces the right number of XML tags. This helps maintain the structured format we want in responses.

2. `soft_format_reward_func`: Provides a "gentle" reward for following the XML format. It might give partial credit for responses that are close to the desired format but not perfect.

3. `strict_format_reward_func`: Enforces rigid adherence to the XML format we saw earlier with the `<reasoning>` and `<answer>` tags. Unlike the soft version, this likely gives an all-or-nothing reward.

4. `int_reward_func`: Likely checks that numerical answers are properly formatted as integers when they should be, which is important for math problems.

5. `correctness_reward_func`: Evaluates if the actual answer is correct, regardless of format.

These reward functions work together like different teachers grading different aspects of a student's work - one checking grammar, another checking math, and so on. The model learns to satisfy all these criteria simultaneously.

When `trainer.train()` runs, it:
1. Takes questions from our dataset
2. Has the model try to answer them
3. Evaluates the answers using all reward functions
4. Adjusts the model's parameters to try to get better rewards next time
5. Repeats this process for the number of steps specified in our training arguments

The trainer uses the configuration we saw earlier to control things like how quickly the model learns (learning rate), how much data it processes at once (batch size), and how long it trains (max_steps).

Think of it like teaching someone to solve math problems while also teaching them to show their work in a specific format. The reward functions provide feedback on both the correctness of the solution and how well they followed the required presentation format.

This approach to training is particularly useful when you want a model to not just give correct answers, but to present them in a consistent, structured way that makes the reasoning process clear and the answers easy to validate.


# Inference


In [ ]:
sampling_params = SamplingParams(
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    max_tokens = CFG.max_tokens
)

This code configures how the model generates text by setting three key parameters that control the balance between creativity and consistency in the model's outputs. Let me explain how each parameter works and why it matters.

The `temperature` parameter acts like a creativity dial for the model. Think of it as controlling how "adventurous" the model is when choosing the next word. A higher temperature (like 0.8 or 1.0) makes the model more willing to take risks and explore different possibilities, which can lead to more diverse and creative outputs. A lower temperature (like 0.2 or 0.1) makes the model more conservative, sticking to the most likely words at each step. This is particularly useful when you want consistent, predictable outputs like for factual responses or coding.

The `top_p` parameter (also known as nucleus sampling) helps control the quality of the generated text by limiting the pool of words the model considers at each step. Imagine sorting all possible next words by their probability and then keeping only the most likely ones that add up to the top_p value (like 0.9 or 0.95). This ensures the model doesn't pick extremely unlikely words while still maintaining some variety in its outputs. It's like having a quality filter that removes the least promising options while keeping enough good choices to maintain interesting outputs.

The `max_tokens` parameter sets a hard limit on how long the generated text can be. Each word or piece of a word the model generates counts as a token. Setting this limit is crucial for several reasons: it prevents the model from rambling indefinitely, helps manage computational resources, and ensures the output stays focused on the task at hand. Think of it as setting a word limit for an essay – it forces the model to be concise and relevant.

These parameters are being pulled from a configuration object (`CFG`), which suggests this is part of a larger system where these values can be easily adjusted without changing the code. This is a good practice because it allows for easy experimentation with different settings to find the optimal balance for your specific use case.

Together, these three parameters give you fine-grained control over the model's text generation behavior, letting you tune it for different tasks – whether you need creative storytelling, precise technical writing, or something in between.

In [ ]:
# raw model

text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)



output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.95it/s, est. speed input: 72.50 toks/s, output: 29.39 toks/s]


'There are two \'r\'s in the word "strawberry".'

This code shows how to directly use a language model to generate a response to a simple question. Let me walk through how it works.

The first step uses the tokenizer's `apply_chat_template` method to format our input in a way the model expects. The input is structured as a list containing a single message dictionary with two key pieces of information:
1. The "role" is set to "user", indicating this is a user's message
2. The "content" is our actual question: "How many r's are in strawberry?"

The `tokenize = False` parameter tells the tokenizer to return the formatted text rather than converting it to token numbers. Setting `add_generation_prompt = True` adds any special tokens or formatting the model needs to know it should generate a response.

Next comes the generation step using `model.fast_generate()`. This method takes a few key inputs:
- A list containing our formatted text (even though we only have one input, it needs to be in a list because the model can handle batch processing)
- The `sampling_params` we configured earlier, which control how the model generates its response
- `lora_request = None` indicates we're using the base model without any LoRA adaptations

The output format from `fast_generate()` is a bit nested, so we need to extract our actual result with `[0].outputs[0].text`. Breaking this down:
- The first `[0]` gets the result for our first (and only) input
- `.outputs[0]` gets the first generated response
- `.text` extracts the actual text content

Finally, the code just displays this output text. This direct approach to using the model gives you more control over the exact process compared to using higher-level training or evaluation interfaces. It's particularly useful when you need to see exactly how the model behaves with specific inputs or when you're debugging the model's behavior.

This kind of direct model interaction is often used as a building block for more complex applications, like chatbots or text analysis tools, where you need precise control over how the model processes each piece of input.

In [ ]:
model.save_lora("grpo_saved_lora")

This code saves the trained LoRA (Low-Rank Adaptation) weights to a directory called "grpo_saved_lora". Let me explain what's happening and why this is important.

When we train a model using LoRA, we're not modifying the full model's weights. Instead, we're creating small, efficient matrices that adapt the model's behavior. Think of it like adding a set of fine-tuning instructions on top of the base model rather than creating an entirely new version. This approach saves enormous amounts of memory and makes the training process much more efficient.

The `save_lora()` method takes these learned adaptations and writes them to disk. This is crucial for several reasons. First, training language models can take significant time and computational resources, so we want to preserve our progress. By saving the LoRA weights, we can reuse them later without having to retrain from scratch.

The saved weights can be used in multiple ways. We might want to:
1. Continue training from this point in the future
2. Apply these adaptations to the same base model on a different machine
3. Share the trained adaptations with others who have access to the base model
4. Compare different versions of training runs to see which produced better results

When someone wants to use these saved weights, they would load them back using a corresponding load function. The beauty of LoRA is that these saved weights are typically very small compared to the full model - often just a few megabytes instead of gigabytes - while still capturing the important adaptations we trained for.

The filename "grpo_saved_lora" indicates these are LoRA weights trained using the GRPO (Gradient-based Role Optimization) method we saw earlier. This naming helps us keep track of which training approach was used to create these weights, which is important when managing multiple different fine-tuning experiments.

It's worth noting that while we're saving just the LoRA weights, we'd still need access to the original base model to use them. Think of it like saving a set of instructions for modifying a recipe - you still need the original recipe to make use of the modifications.

In [ ]:
# tuned model
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)


output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.77s/it, est. speed input: 15.51 toks/s, output: 79.01 toks/s]


'<reasoning>\nTo determine how many r\'s are in the word "strawberry," I will go through the word character by character and count the instances of the letter \'r\'.\n1. The first character is \'s\', not an \'r\'.\n2. The second character is \'t\', not an \'r\'.\n3. The third character is \'r\', this is the first \'r\'.\n4. The fourth character is \'w\', not an \'r\'.\n5. The fifth character is \'a\', not an \'r\'.\n6. The sixth character is \'r\', this is the second \'r\'.\n7. The seventh character is \'r\', this is the third \'r\'.\n8. The eighth character is \'b\', not an \'r\'.\n9. The ninth character is \'b\', not an \'r\'.\n10. The tenth character is \'y\', not an \'r\'.\nTherefore, there are 3 instances of the letter \'r\' in the word "strawberry."\n</reasoning>\n<answer>\n3\n</answer>'

This code demonstrates how to use our fine-tuned model to generate a structured response using the LoRA weights we just saved. Let me explain how this process differs from our previous example and why each step matters.

The first key difference is in how we construct the input. We're now using a more sophisticated prompt structure that includes both a system message and a user message. The system message contains our `SYSTEM_PROMPT` which we saw earlier - this is the template that instructs the model to format its response with separate `<reasoning>` and `<answer>` sections. By including this as a system message, we're effectively giving the model its "instructions" for how to approach the task.

The tokenizer's `apply_chat_template` method takes this two-part message structure and formats it appropriately for our model. Just like before, we set `tokenize = False` to get text output rather than tokens, and `add_generation_prompt = True` to signal that the model should generate a response.

The most significant change comes in the generation step. While we're still using `model.fast_generate()`, we're now adding a crucial new component: `lora_request = model.load_lora("grpo_saved_lora")`. This line loads the LoRA weights we previously saved and applies them to our model during generation. Think of it like loading a set of specialized training that teaches the model to follow our desired response format.

The output extraction follows the same pattern as before, accessing the first generated response's text. However, because we've loaded our LoRA weights and included the system prompt, we should expect the output to follow our structured format with explicit reasoning and answer sections, rather than just providing a direct answer like the untuned model might.

This approach showcases how we can take a general-purpose language model and adapt it to follow specific formatting and reasoning patterns without having to retrain the entire model. The LoRA weights act like a lightweight set of instructions that guide the model to structure its thoughts and responses in our desired format, making its reasoning process more transparent and its outputs more consistent.

This kind of structured output is particularly valuable when you need to automatically process or validate the model's responses, as the clear XML format makes it easy to extract and verify both the reasoning steps and the final answer.